In [ ]:
import wandb
wandb.init()


wandb: Currently logged in as: kkamalesh (use `wandb login --relogin` to force relogin)


In [ ]:
import  os
import re
import  pandas as pd
import  numpy as np
import  torch
import torch.nn as nn
import  torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.nn.utils import weight_norm
import torch.optim as optim
import string 
import math
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
#from sklearn.preprocessing import LabelEncoder
import gensim.downloader as api
from gensim.models import KeyedVectors
from torch.utils.data import DataLoader
import random
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight
from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.chdir('/content/drive/MyDrive/data')
train_data=pd.read_csv('train.csv').to_numpy()
np.random.shuffle(train_data)

In [ ]:
classes=np.unique(train_data[:,3])
label=train_data[:,3]


In [ ]:
class_weight=torch.tensor(compute_class_weight('balanced',classes=classes,y=label)).type(torch.FloatTensor)

In [ ]:
#torch.cuda.is_available()

In [ ]:
#Counter(train_data[:,3])

In [ ]:
#glove=KeyedVectors.load_word2vec_format('/kaggle/input/glove300/glove_300.txt',binary=False)

In [ ]:
train_data=np.column_stack((train_data[:,0]+' '+train_data[:,1],train_data[:,3]))

In [ ]:
sample=pd.read_csv('train.csv').to_numpy()

In [ ]:
stop_words=set(stopwords.words('english'))
lemmatizer=WordNetLemmatizer()
for i,data in enumerate(train_data[:,0]):
    data=data.lower()  
    data=re.sub(r'\d+','',data) 
    data=data.translate(str.maketrans('','', string.punctuation)) 
    data=[data.strip()]
    data=word_tokenize(data[0]) 
    data=[y for y in  data if not y in stop_words]
    data=[lemmatizer.lemmatize(y) for y in data] 
    train_data[i,0]=data

In [ ]:
#train_data=np.array(sorted(train_data,key=lambda x:len(x[0])))

In [ ]:
sample2=train_data

In [ ]:
length=train_data.shape[0]
description,target_ind=np.split(train_data,2,axis=1)
description=np.reshape(description,(length,))  # array of lists
target_ind=np.reshape(target_ind,(length,))

In [ ]:
description_train,description_test,target_train,target_test=train_test_split(description,target_ind,test_size=0.125,shuffle=False)
len_train=len(target_train)
len_test=len(target_test)

In [ ]:
test_data=pd.read_csv('test.csv').to_numpy()
test_data.shape

(8106, 3)

In [ ]:
glove=api.load('glove-wiki-gigaword-300')

In [ ]:
EMB_DIM=300
#BATCH_SIZE=64

In [ ]:
class  word_embedding(nn.Module):
    
    def __init__(self,token,emb_dim):  #here token is a list

        super(word_embedding,self).__init__()  
        self.emb_dim=emb_dim
        self.vocab_length=len(token)
        self.weight_matrix=np.zeros((self.vocab_length,self.emb_dim))
        self.emd_layer=nn.Embedding(self.vocab_length,self.emb_dim)
        check_word=np.array([1 if word in glove.vocab else 0 for word in token])
        glove_avg=np.zeros(emb_dim)
        if np.count_nonzero(check_word)==0:
          self.weight_matrix=np.random.normal(size=(self.vocab_length,self.emb_dim))
        else:
          for i in np.argwhere(check_word==1):
            word=token[i[0]]
            glove_avg=glove_avg+glove[word]
            self.weight_matrix[i,:]=glove[word]
          glove_avg=glove_avg/np.count_nonzero(check_word)
          for i in np.argwhere(check_word==0):
            self.weight_matrix[i,:]=glove_avg

            '''
            try:
                self.weight_matrix[i,:]=glove[word]
            except KeyError:
                std_d=math.sqrt(2/emb_dim)  #he initialisation
                self.weight_matrix[i,:]=np.random.normal(scale=std_d,size=(1,emb_dim))'''
        self.emd_layer.load_state_dict({'weight':torch.from_numpy(self.weight_matrix)})             
              
    def forward(self):
        
        token_tensor=torch.arange(self.vocab_length)
        out=self.emd_layer(token_tensor).reshape((1,self.emb_dim,self.vocab_length))
        #out.requires_grad=False
        return out

In [ ]:
#np.count_nonzero(np.array([1212,323,4343]))

In [ ]:
def collate_fn(token):
  emb=word_embedding(token[0],EMB_DIM)()
  emb=emb.detach()
  if torch.cuda.is_available():
    emb=emb.cuda()
  return emb      

In [ ]:
description_train=list(DataLoader(description_train,collate_fn=collate_fn))


In [ ]:
description_test=list(DataLoader(description_test,collate_fn=collate_fn))

In [ ]:
target_train=list(DataLoader(target_train,collate_fn=None))
if torch.cuda.is_available():
    target_train=[x.cuda() for x in target_train]

In [ ]:
target_test=list(DataLoader(target_test,collate_fn=None))
if torch.cuda.is_available():
    target_test=[x.cuda() for x in target_test]

In [ ]:
class trunc1d(nn.Module):
    
    def __init__(self,padding):
        
        super(trunc1d,self).__init__()
        self.padding=padding
        
    def forward(self,x):
        
        return x[:,:,:-self.padding].contiguous()        

In [ ]:
class TemporalConvLayer(nn.Module):
    
    def __init__(self,in_channels,out_channels,kernel_size,padding,dil_factor,stride=1,dropout=0.5):
        
        super(TemporalConvLayer,self).__init__()
        self.conv1=weight_norm(nn.Conv1d(in_channels,out_channels,kernel_size,stride=stride,padding=padding,dilation=dil_factor))
        self.conv2=weight_norm(nn.Conv1d(out_channels,out_channels,kernel_size,stride=stride,padding=padding,dilation=dil_factor))
        self.trunc=trunc1d(padding)
        self.net=nn.Sequential(
        self.conv1,
        self.trunc,
        nn.ReLU(),
        nn.Dropout(dropout),
        self.conv2,
        self.trunc,
        nn.ReLU(),
        nn.Dropout(dropout))
        self.scale_input= nn.Conv1d(in_channels,out_channels,1) if in_channels!=out_channels else None
        self.weight_initialise()
        
    def weight_initialise(self):
        
        nn.init.kaiming_normal_(self.conv1.weight)
        nn.init.kaiming_normal_(self.conv2.weight)
        if self.scale_input is not None:
            nn.init.kaiming_normal_(self.scale_input.weight)
    
    def forward(self,x):
        
        residual=x if self.scale_input is None else self.scale_input(x)
        relu=nn.ReLU()
        return relu(residual+self.net(x)) #con layer

In [ ]:
class TemporalConvNetwork(nn.Module):
    
    def __init__(self,input_channels,channel_arr,kernel_size=4,dropout=0.5):
        
        super(TemporalConvNetwork,self).__init__()
        tc_layers=[]   #temporal convolution layer
        num_layers=len(channel_arr)
        for i in range(num_layers):
            in_channels=input_channels if i==0 else channel_arr[i-1]
            out_channels=channel_arr[i]
            dil_factor=2**i
            padding=(kernel_size-1)*dil_factor
            tc_layers+=[TemporalConvLayer(in_channels,out_channels,kernel_size,stride=1,dil_factor=dil_factor,dropout=dropout,padding=padding)]
            
        self.net=nn.Sequential(*tc_layers)
    
    def forward(self,x):
        
        return self.net(x)     #con network   

In [ ]:
class GatedTempConvNet(nn.Module):
    
    def  __init__(self,input_channels,channel_arr,kernel_size=4,dropout=0.5):
        
        super(GatedTempConvNet,self).__init__()
        num_layers=len(channel_arr)
        self.output_channels=channel_arr[num_layers-1]
        self.net1=nn.Sequential(TemporalConvNetwork(input_channels,channel_arr,kernel_size=4,dropout=0.5),nn.ReLU())
        self.net2=TemporalConvNetwork(input_channels,channel_arr,kernel_size=4,dropout=0.5)
        self.scale_input=nn.Conv1d(input_channels,self.output_channels,kernel_size=1) if  input_channels!=self.output_channels else None
        if self.scale_input is not None:
            nn.init.xavier_normal_(self.scale_input.weight)
            
    def forward(self,x):
        
        tcn1=self.net1(x)
        tcn2=self.net2(x)
        residual=x if self.scale_input is None else self.scale_input(x)
        y=residual+(tcn1*tcn2)
        return y    #gated

In [ ]:
class Bidirection(nn.Module):
    
    def __init__(self,input_channels,channel_arr,kernel_size=4,dropout=0.5):
        
        super(Bidirection,self).__init__()
        self.original_seq=GatedTempConvNet(input_channels,channel_arr,kernel_size=4,dropout=0.5)
        self.reverse_seq=GatedTempConvNet(input_channels,channel_arr,kernel_size=4,dropout=0.5)
        
    def forward(self,x):
        
        x_forward=self.original_seq(x)
        x_reverse=self.reverse_seq(torch.flip(x,dims=[-1]))
        out=torch.cat((x_forward,x_reverse),dim=1)
        return out #bidirection

In [ ]:
class AttentionLayer(nn.Module):
    
    def __init__(self,in_channels,out_channels):
        
        super(AttentionLayer,self).__init__()
        self.conv1=nn.Conv1d(in_channels,out_channels,kernel_size=1)
        self.conv2=nn.Conv1d(out_channels,1,kernel_size=1)
        self.net=nn.Sequential(
        self.conv1,
        nn.Tanh(), 
        nn.Dropout(p=0.5),
        self.conv2,
        nn.Softmax(dim=-1))
        self.weight_initialise()
    
    def  weight_initialise(self):
        
        nn.init.xavier_normal_(self.conv1.weight)
        nn.init.xavier_normal_(self.conv2.weight)
    
    def forward(self,x):
        
        output=torch.sum(x*self.net(x),dim=-1).reshape(x.shape[0],x.shape[1])
        return output    #attention

In [ ]:
class Classification(nn.Module):
    
    def __init__(self,input_dim,num_classes,output_nodes,num_layers,dropout=0.7):
        
        super(Classification,self).__init__()
        layers=[]
        for i in range(num_layers-1):
            in_features=input_dim if i==0 else output_nodes
            model=nn.Linear(in_features,output_nodes)
            nn.init.kaiming_normal_(model.weight)
            layers.append(model)
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
        final_class=nn.Linear(output_nodes,num_classes)
        nn.init.xavier_normal_(final_class.weight)
        layers.append(final_class)
        self.net=nn.Sequential(*layers)
    
    def forward(self,x):
        return self.net(x)  #classification

In [ ]:
class Model(nn.Module):
    
    def __init__(self,num_classes,output_nodes,num_layers,att_channels,input_channels,channel_arr,kernel_size=4,dropout=0.5):
        
        super(Model,self).__init__()
        channel_dim=channel_arr[-1]
        self.bidirection=Bidirection(input_channels,channel_arr,kernel_size,dropout)
        self.attention=AttentionLayer(2*channel_dim,att_channels)
        self.classification=Classification(2*channel_dim,num_classes,output_nodes,num_layers,dropout=0.7)
        self.net=nn.Sequential(
        self.bidirection,
        self.attention,
        self.classification)
        
    def forward(self,x):
        
        return self.net(x)   #model

In [ ]:
EPOCHS=25
softmax=nn.Softmax(dim=1)   #do sanity check
model=Model(500,128,2,64,EMB_DIM,[64,64],kernel_size=4,dropout=0.5)
if torch.cuda.is_available():
    model.cuda()
    class_weight=class_weight.cuda()
loss_fun=nn.CrossEntropyLoss(weight=class_weight)
optimizer=optim.Adam(model.parameters(),lr=1e-3,weight_decay=1e-3)
ls1=[]
ls2=[]
loss_history_train=[]
loss_history_test=[]
for epoch in range(EPOCHS):
    class_pred=[]
    loss_train=0
    model.train()
    for i,sample in enumerate(description_train):
        target=target_train[i]
        out=model(sample)
        loss=loss_fun(out,target)
        out_target=torch.argmax(softmax(out),dim=1)
        #ls1.append(out_target-target_batch)
        #ls2.append(out_target)
        class_pred.append(out_target)
        optimizer.zero_grad()
        loss.backward()# if epoch!=0 else loss.backward(retain_graph=True)
        optimizer.step()
        loss_train+=loss.item()
        #print(loss.item())
    loss_train=loss_train/len_train
    loss_history_train.append(loss_train)
    correct_pred=len_train-torch.count_nonzero(torch.tensor(class_pred)-torch.tensor(target_train)).item()
    train_acc=correct_pred/len_train
    model.eval()
    loss_test=0
    correct_pred=0
    with torch.no_grad():
        for i,sample in enumerate(description_test):
            target=target_test[i]
            out=model(sample)
            loss=loss_fun(out,target)
            loss_test+=loss.item()
            out_target=torch.argmax(softmax(out))
            ls1.append(out_target-target)
            ls2.append(out_target)
            correct_pred+=1 if (out_target-target).item()==0 else 0
        loss_test=loss_test/len_test
        loss_history_test.append(loss_test)
        test_acc=correct_pred/len_test
    
    print('epoch',epoch,'train_loss %.4f'%loss_train,'test_loss %.4f'%loss_test,'train_acc',train_acc,'test_acc',test_acc)
    if (epoch%5==0 and epoch>=5):
      checkpoint={'model':Model(500,512,6,512,EMB_DIM,[1024,1024,1024],kernel_size=4,dropout=0.5),
           'state_dict':model.state_dict(),
           'optimizer': optimizer.state_dict()}
      torch.save(checkpoint,'/content/drive/MyDrive/data/checkpoint{}.pth'.format(epoch))

    
plt.figure()
plt.plot(np.arange(EPOCHS),loss_history_test)
plt.plot(np.arange(EPOCHS),loss_history_train)
plt.legend(['test','train'])
plt.show()

        
        
        
        
        
    